In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18
from torchvision.ops import MultiScaleRoIAlign, RoIPool

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.custom_head = nn.Sequential(
            nn.Linear(512 * block.expansion, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
        )
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.custom_head(x)
        x = self.fc(x)

        return x

In [ ]:
def resnet18_custom(num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

In [ ]:
class RPN(nn.Module):
    def __init__(self, anchor_generator, in_channels, hidden_channels):
        super(RPN, self).__init__()
        self.conv = nn.Conv2d(in_channels, hidden_channels, kernel_size=3, stride=1, padding=1)
        self.cls_logits = nn.Conv2d(hidden_channels, anchor_generator.num_anchors_per_location()[0] * 2, kernel_size=1)
        self.bbox_pred = nn.Conv2d(hidden_channels, anchor_generator.num_anchors_per_location()[0] * 4, kernel_size=1)
        self.anchor_generator = anchor_generator

    def forward(self, features):
        t = F.relu(self.conv(features))
        cls_logits = self.cls_logits(t)
        bbox_pred = self.bbox_pred(t)
        return cls_logits, bbox_pred

class DetectionHead(nn.Module):
    def __init__(self, in_features, num_classes):
        super(DetectionHead, self).__init__()
        self.fc = nn.Linear(in_features, num_classes)
    
    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

class FasterRCNN(nn.Module):
    def __init__(self, num_classes):
        super(FasterRCNN, self).__init__()
        # Backbone
        backbone = resnet18(pretrained=True)  # Using pre-defined ResNet-18
        self.backbone = nn.Sequential(*list(backbone.children())[:-2])  # Exclude the last fc layer
        out_channels = backbone.fc.in_features  # The feature size expected by the RPN

        # RPN (Simplified Version)
        self.rpn = RPN(anchor_generator=SomeAnchorGenerator(), in_channels=out_channels, hidden_channels=256)
        
        # RoI Pooling
        self.roi_pooler = MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)
        
        # Detection Head
        self.det_head = DetectionHead(in_features=out_channels * 7 * 7, num_classes=num_classes)

    def forward(self, images, targets=None):
        features = self.backbone(images)
        
        # RPN Forward Pass
        rpn_cls_logits, rpn_bbox_pred = self.rpn(features)
        
        # Here you would generate proposals from the RPN outputs and apply NMS
        # proposals = ...
        
        # Apply RoI Pooling
        # pooled_features = self.roi_pooler(features, proposals, image_shapes)
        
        # Detection Head Forward Pass
        # detection_classes = self.det_head(pooled_features)
        
        # For simplicity, this part is left out. It involves matching proposals with ground truth,
        # calculating losses, and decoding final bounding boxes and class scores.
        
        return dict(rpn_cls_logits=rpn_cls_logits, rpn_bbox_pred=rpn_bbox_pred)

# Placeholder for an anchor generator - this requires an actual implementation
class SomeAnchorGenerator:
    def num_anchors_per_location(self):
        return [3]  # Placeholder value, typically depends on the actual anchor scales and aspect ratios
